In [41]:
import glob
import os
import yaml
import pandas as pd

In [42]:
def open_sample_sheet(sample_sheet_fp, lanes=False, skiprows=18):
    """Read in an IGM sample sheet and return a pandas DF with primary data table"""
    sample_sheet = pd.read_excel(sample_sheet_fp, skiprows = skiprows, header=1)
    
    # Remove trailing whitespace from column names and homogenize case
    sample_sheet.columns = [x.strip().lower() for x in sample_sheet.columns]
    
    if lanes:
        sample_sheet = sample_sheet.loc[sample_sheet['lane'].isin(lanes)]
    if 'sample name' not in sample_sheet.columns and 'sample_id' in sample_sheet.columns:
        sample_sheet['sample name'] = sample_sheet['sample_id']

    return(sample_sheet)

In [43]:
def open_sequencing_manifest(manifest_fp, lanes='False', sheetname='Sample Information', skiprows=18):
    """Read in an IGM sequencing manifest and return a pandas DF with primary data table"""
    sample_sheet = pd.read_excel(manifest_fp, sheetname=sheetname, skiprows=skiprows, header=1)
    
    # Remove trailing whitespace from column names and homogenize case
    sample_sheet.columns = [x.strip().lower() for x in sample_sheet.columns]
    
    if lanes:
        sample_sheet = sample_sheet.loc[sample_sheet['lane'].isin(lanes)]
        
    return(sample_sheet)

In [44]:
def get_read(sample, seq_dir, read):
    """Function to pull a given read based on sample name from the reads directory"""
    reads = glob.glob(os.path.join(seq_dir, "{0}_*_{1}_*.fastq.gz".format(sample, read)))
    if len(reads) == 1:
        return(reads[0])
    elif len(reads) > 1:
        raise ValueError('Too many reads found for {0} in {1}:\n'
                         'read_str: {2}'.format(sample, seq_dir, reads))
    elif len(reads) < 1:
        raise ValueError('Too few reads found for {0} in {1}:\n'
                         'read_str: {1}\n'.format(sample, seq_dir, reads))   

In [45]:
def make_sample_dict_pe(sample_sheet, seq_dir,
                        forward = 'R1',
                        reverse = 'R2',
                        adaptor = '$CONDA_ENV_PATH/share/trimmomatic-*/adapters/TruSeq3-PE-2.fa',
                        phred = 'phred33',
                        sample_header = 'Sample_Prefix',
                        sample_name = 'Sample',
                        tolerate_losses = True):
    
    samples_pe = {'samples_pe': {}}
    for x in sample_sheet.index:
        try:
            samples_pe['samples_pe'][sample_sheet.loc[x, sample_name]] = {
               'forward': get_read(sample_sheet.loc[x, sample_header], seq_dir, forward),
               'reverse': get_read(sample_sheet.loc[x, sample_header], seq_dir, reverse),
               'adaptor': adaptor,
               'phred': phred
            }
        except ValueError:
            if tolerate_losses:
                print('No sequences found for %s' % sample_sheet.loc[x, sample_name])
            else:
                raise ValueError
    
    return(samples_pe)

In [46]:
def make_sample_dict_se(sample_sheet, seq_dir,
                        forward = 'R1',
                        adaptor = '$CONDA_ENV_PATH/share/trimmomatic-*/adapters/TruSeq3-PE-2.fa',
                        phred = 'phred33',
                        sample_header = 'Sample_Prefix',
                        sample_name = 'Sample',
                        tolerate_losses = True):
    
    samples_pe = {'samples_pe': {}}
    for x in sample_sheet.index:
        try:
            samples_pe['samples_pe'][sample_sheet.loc[x, sample_name]] = {
               'forward': get_read(sample_sheet.loc[x, sample_header], seq_dir, forward),
               'adaptor': adaptor,
               'phred': phred
            }
        except ValueError:
            if tolerate_losses:
                print('No sequences found for %s' % sample_sheet.loc[x, sample_name])
            else:
                raise ValueError
    
    return(samples_pe)

In [47]:
def format_yaml_pe(RUN, samples_pe,
                   TMP_DIR_ROOT = '/localscratch',
                   host_db = '/home/jgsanders/ref_data/genomes/Homo_sapiens_Bowtie2_v0.1/Homo_sapiens',
                   envs = {'KNEAD_ENV': 'source activate kneaddata',
                           'BOWTIE_ENV': 'source activate kneaddata',
                           'QC_ENV': 'source activate kneaddata',
                           'TRIM_ENV': 'source activate kneaddata',
                           'HUMANN2_ENV': 'source activate kneaddata',
                           'METAPHLAN_ENV': 'source activate kneaddata'},
                   software = {'gzip': 'gzip',
                               'trimmomatic': 'trimmomatic'},
                   params = {'TRIMMOMATIC': {'QUAL': 'LEADING:20 TRAILING:20 AVGQUAL:30 MINLEN:32 TOPHRED33',
                                              'ILLUMINACLIP': '2:30:10'},
                             'HUMANN2': {'METAPHLAN_DIR': '/home/jgsanders/share/metaphlan2',
                                         'HUMANN2_NT_DB': '/home/jgsanders/ref_data/humann2/chocophlan',
                                         'HUMANN2_AA_DB': '/home/jgsanders/ref_data/humann2/uniref',
                                         'NORMS': ['cpm','relab'],
                                         'OTHER': ''}},
                   default_flow_style = False):
    
    config_str = ''

    config_str += yaml.dump({'TMP_DIR_ROOT': TMP_DIR_ROOT}, default_flow_style = default_flow_style)

    config_str += yaml.dump({'RUN': RUN}, default_flow_style = default_flow_style)
    
    config_str += yaml.dump({'HOST_DB': host_db}, default_flow_style = default_flow_style)
    
    config_str += yaml.dump({'ENVS': envs}, default_flow_style = default_flow_style)            
    
    config_str += yaml.dump({'SOFTWARE': software}, default_flow_style = default_flow_style)

    config_str += yaml.dump({'PARAMS': params}, default_flow_style = default_flow_style)

    config_str += yaml.dump(samples_pe, default_flow_style = default_flow_style)
    
    return(config_str)

In [48]:
def format_yaml_se(RUN, samples_se,
                   TMP_DIR_ROOT = '/localscratch',
                   host_db = '/home/jgsanders/ref_data/genomes/Homo_sapiens_Bowtie2_v0.1/Homo_sapiens',
                   envs = {'KNEAD_ENV': 'source activate kneaddata',
                           'BOWTIE_ENV': 'source activate kneaddata',
                           'QC_ENV': 'source activate kneaddata',
                           'TRIM_ENV': 'source activate kneaddata',
                           'HUMANN2_ENV': 'source activate kneaddata',
                           'METAPHLAN_ENV': 'source activate kneaddata'},
                   software = {'gzip': 'gzip',
                               'trimmomatic': 'trimmomatic'},
                   params = {'TRIMMOMATIC': {'QUAL': 'LEADING:20 TRAILING:20 AVGQUAL:30 MINLEN:32 TOPHRED33',
                                              'ILLUMINACLIP': '2:30:10'},
                             'HUMANN2': {'METAPHLAN_DIR': '/home/jgsanders/share/metaphlan2',
                                         'HUMANN2_NT_DB': '/home/jgsanders/ref_data/humann2/chocophlan',
                                         'HUMANN2_AA_DB': '/home/jgsanders/ref_data/humann2/uniref',
                                         'NORMS': ['cpm','relab'],
                                         'OTHER': ''}},
                   default_flow_style = False):
                
    config_str = ''

    config_str += yaml.dump({'TMP_DIR_ROOT': TMP_DIR_ROOT}, default_flow_style = default_flow_style)

    config_str += yaml.dump({'RUN': RUN}, default_flow_style = default_flow_style)
    
    config_str += yaml.dump({'HOST_DB': host_db}, default_flow_style = default_flow_style)
    
    config_str += yaml.dump({'ENVS': envs}, default_flow_style = default_flow_style)            
    
    config_str += yaml.dump({'SOFTWARE': software}, default_flow_style = default_flow_style)

    config_str += yaml.dump({'PARAMS': params}, default_flow_style = default_flow_style)

    config_str += yaml.dump(samples_se, default_flow_style = default_flow_style)
    
    return(config_str)

# Read in the sample data 

We ultimately need a Pandas DataFrame that has the following two columns: `Sample` and `Sample Prefix`. `Sample` has the name you'd like to call the sample on the filesystem; a good candidate would be the sample's name in the metadata mapping file you'll use later. `Sample Prefix` is what the script will use to locate the Fastq file in the sequencing directory. This should match exactly the beginning of the Fastq filename for that sample. 

Note that for the time being, you will want to make sure that you have a unique 1:1 relationship between `Sample` and `Sample Prefix`: if for example you sequence the same sample multiple times on a single run (for example on different lanes, or using different library prep parameters), we don't currently handle that automatically.

In the latter case, one option would be to prepare multiple config.yaml files using this notebook for each of the condition sets you use, with different `Run` names. Presuming you have four original samples (say `sample1`, `sample2`...) and each were sequenced with two preps on the same lane (giving file names like `sample1_prep1_L001_R1.fastq.gz`), your final `sample_sheet` dataframe for the first run would look like this:

In [49]:
run1 = pd.DataFrame({'Sample': ['sample1', 'sample2', 'sample2', 'sample4'],
                     'Sample Prefix': ['sample1_prep1', 'sample2_prep1', 'sample2_prep1', 'sample4_prep1']})
run1

,Sample,Sample Prefix
0,sample1,sample1_prep1
1,sample2,sample2_prep1
2,sample2,sample2_prep1
3,sample4,sample4_prep1


You would then make a second `sample_sheet` DataFrame for the prep2 sequence files:

In [50]:
run2 = pd.DataFrame({'Sample': ['sample1', 'sample2', 'sample2', 'sample4'],
                     'Sample Prefix': ['sample1_prep2', 'sample2_prep2', 'sample2_prep2', 'sample4_prep2']})
run2

,Sample,Sample Prefix
0,sample1,sample1_prep2
1,sample2,sample2_prep2
2,sample2,sample2_prep2
3,sample4,sample4_prep2


You would make a different config.yaml file for each of these runs, and the resulting data directory after running snakemake would have four sample directories (`sample1` through `sample4`), each with two run directories (`Run1` and `Run2`). 

I have provided two methods, `open_sequencing_manifest` and `open_sample_sheet`, for simplifying construction of the `sample_sheet` dataframe from the Excel files provided by IGM. The 'sequence manifest' is a two-sheet Excel workbook, with sample information on the second sheet. The 'sample sheet' is a one-sheet workbook. In both cases, the precise line number where the sample information starts is sometimes variable. Load with the appropriate method, and adjust the `skiprows` number until you have header information properly loaded as the column names of the displayed `sample_sheet` workbook.

Either way, you want to end up with a column called `sample name`. 

In [51]:
# this is the Excel document provided by IGM

# If you only want samples from some lanes, you can provide a list of those lane numbers here
lanes = [1]

# This is the path to the folder on Barnacle where the raw reads are located
seq_dir = 'example/reads/Run1/'

# Read in the sample manifest
sequencing_manifest_fp = 'example/reads/example_sample_manifest.xlsx'
sample_sheet = open_sequencing_manifest(sequencing_manifest_fp, lanes=lanes, skiprows=20)

sample_sheet

,sample name,sample code,library size (bp),library prep method,index 1 (name),index 1 (sequence),index 2 (name),index 2 (sequence),conc. (nm),volume (μl),quantification method,lane
0,sample1,1,400-600,KapaHyperPlus,D001,ATCTAGCCGGCC,NaN,NaN,NaN,NaN,Pico,1
1,sample2,2,400-600,KapaHyperPlus,D013,AAGCGTACGTCC,NaN,NaN,NaN,NaN,Pico,1


In [52]:
# Alternatively, read in the sample sheet
sequencing_sample_sheet_fp = 'example/reads/example_sample_sheet.xls'
sample_sheet = open_sample_sheet(sequencing_sample_sheet_fp, lanes=lanes, skiprows=18)

sample_sheet

,lane,sample_id,sample_name,sample_plate,sample_well,i7_index_id,index,sample_project,description,sample name
0,1,sample1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sample1
1,1,sample2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sample2


Sometimes there are inconsistencies in the sample naming between the ultimate Fastq filenames and the sample names given in the sample sheet -- for example, periods or spaces are replaced by underscores. 

Here, we'll use the `sample name` column to create the `Sample` and `Sample Prefix` columns we need for the next steps. At this stage, we can use the Pandas `replace` functionality to make these kinds of substitutions as needed. Also note that any numerical sample names need to be stored as string types.

In [53]:
# These columns can be modified with regexes to fix sample names or add common sample prefixes if necessary
# sample_sheet['Sample_Prefix'] = sample_sheet['Sample Name'].replace('\.','_', regex=True).astype(str)
# sample_sheet['Sample'] = sample_sheet['Sample Name'].replace('\.','_', regex=True).astype(str)
sample_sheet['Sample Prefix'] = sample_sheet['sample name'].astype(str)
sample_sheet['Sample'] = sample_sheet['sample name'].astype(str)

sample_sheet

,lane,sample_id,sample_name,sample_plate,sample_well,i7_index_id,index,sample_project,description,sample name,Sample Prefix,Sample
0,1,sample1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sample1,sample1,sample1
1,1,sample2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sample2,sample2,sample2


Make samples dictionary
------

This links the sample name to the original location of the forward (and reverse) reads. You'll want to make sure to use a path to the correct adapter file for your library prep. (The ones in my home directory should be readable and ok to use on Barnacle.)

To instead use single ended reads, call `samples_se()` instead.

In [54]:
# Nextera adapter path: '/home/jgsanders/git_sw/git_bin/Trimmomatic-0.36/adapters/NexteraPE-PE.fa'

samples_pe = make_sample_dict_pe(sample_sheet, seq_dir,
                        adaptor = '/home/jgsanders/git_sw/git_bin/Trimmomatic-0.36/adapters/TruSeq3-PE-2.fa',
                        forward = 'R1',
                        reverse = 'R2',
                        sample_header = 'Sample Prefix',
                        sample_name = 'Sample')

samples_pe

{'samples_pe': {'sample1': {'adaptor': '/home/jgsanders/git_sw/git_bin/Trimmomatic-0.36/adapters/TruSeq3-PE-2.fa',
   'forward': 'example/reads/Run1/sample1_S312_R1_L001.fastq.gz',
   'phred': 'phred33',
   'reverse': 'example/reads/Run1/sample1_S312_R2_L001.fastq.gz'},
  'sample2': {'adaptor': '/home/jgsanders/git_sw/git_bin/Trimmomatic-0.36/adapters/TruSeq3-PE-2.fa',
   'forward': 'example/reads/Run1/sample2_S521_R1_L001.fastq.gz',
   'phred': 'phred33',
   'reverse': 'example/reads/Run1/sample2_S521_R2_L001.fastq.gz'}}}

Format YAML file
================

Make the actual config file using the methods provided.

You will want to modify the `RUN` variable with a good name for your particular run.


Important parameter information
-------------------------------
You may also want to pass optional parameters to modify execution. The optional parameters and their defaults are:

```
TMP_DIR_ROOT = '/localscratch',
host_db = '/home/jgsanders/ref_data/genomes/Homo_sapiens_Bowtie2_v0.1/Homo_sapiens',
envs = {'KNEAD_ENV': 'source activate kneaddata',
       'BOWTIE_ENV': 'source activate kneaddata',
       'QC_ENV': 'source activate kneaddata',
       'TRIM_ENV': 'source activate kneaddata',
       'HUMANN2_ENV': 'source activate kneaddata',
       'METAPHLAN_ENV': 'source activate kneaddata'},
software = {'gzip': 'gzip',
           'trimmomatic': 'trimmomatic'},
params = {'TRIMMOMATIC': {'QUAL': 'LEADING:20 TRAILING:20 AVGQUAL:30 MINLEN:32 TOPHRED33',
                          'ILLUMINACLIP': '2:30:10'},
         'HUMANN2': {'METAPHLAN_DIR': '/home/jgsanders/share/metaphlan2',
                     'HUMANN2_NT_DB': '/home/jgsanders/ref_data/humann2/chocophlan',
                     'HUMANN2_AA_DB': '/home/jgsanders/ref_data/humann2/uniref',
                     'NORMS': ['cpm','relab'],
                     'OTHER': ''}},
```

These are provided as separate keyword argument dicts or strings to the `format_yaml` function.


**In particular, make sure you provide:**
1. the correct conda environment for the `envs` variables
2. the correct `host_db` Bowtie filename base (this is the path up to the extensions, not an actual filename; for example, in the default, the directory `/home/jgsanders/ref_data/genomes/Homo_sapiens_Bowtie2_v0.1/` has files in it like `Homo_sapiens.1.bt2` and `Homo_sapiens.rev.1.bt2`)
3. a valid Trimmomatic executable filepath. If you have Trimmomatic installed in your `TRIM_ENV` environment, you can just use `trimmomatic`.

Other parameters notes:
- For very large sequence files, you can save significant time by installing [pigz](http://www.zlib.net/pigz/) and passing that as the gzip executable. 
- The HUMAnN2 Uniref database provided as default from my Home directory on Barnacle is the EC-filtered UniRef90 database. For less well-characterized metagenomes you may want to use UniRef50, at `/home/jgsanders/ref_data/humann2/uniref50`. 
- The `NORMS` parameter is a list of normalization types to pass to HUMAnN2. I typically only use CPM, but by default we will generate both. 
- Any additional HUMAnN2 command-line parameters can be passed via 'OTHER'. 

In [55]:
RUN = 'Run1'
config_str = format_yaml_pe(RUN, samples_pe, host_db = '/home/jgsanders/ref_data/genomes/mouse/mouse')
with open('config_%s.yaml' % RUN, 'w') as f:
    f.write(config_str)

In [56]:
print(config_str)

TMP_DIR_ROOT: /localscratch
RUN: Run1
HOST_DB: /home/jgsanders/ref_data/genomes/mouse/mouse
ENVS:
  BOWTIE_ENV: source activate kneaddata
  HUMANN2_ENV: source activate kneaddata
  KNEAD_ENV: source activate kneaddata
  METAPHLAN_ENV: source activate kneaddata
  QC_ENV: source activate kneaddata
  TRIM_ENV: source activate kneaddata
SOFTWARE:
  gzip: gzip
  trimmomatic: trimmomatic
PARAMS:
  HUMANN2:
    HUMANN2_AA_DB: /home/jgsanders/ref_data/humann2/uniref
    HUMANN2_NT_DB: /home/jgsanders/ref_data/humann2/chocophlan
    METAPHLAN_DIR: /home/jgsanders/share/metaphlan2
    NORMS:
    - cpm
    - relab
    OTHER: ''
  TRIMMOMATIC:
    ILLUMINACLIP: '2:30:10'
    QUAL: LEADING:20 TRAILING:20 AVGQUAL:30 MINLEN:32 TOPHRED33
samples_pe:
  sample1:
    adaptor: /home/jgsanders/git_sw/git_bin/Trimmomatic-0.36/adapters/TruSeq3-PE-2.fa
    forward: example/reads/Run1/sample1_S312_R1_L001.fastq.gz
    phred: phred33
    reverse: example/reads/Run1/sample1_S312_R2_L001.fastq.gz
  sample2:
    a